In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

In [3]:
from langchain_core.messages import AIMessage, HumanMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Bedanta and I am a Chief AI Engineer"),
         AIMessage(content="Hello Bedanta! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Bedanta, and you are a Chief AI Engineer!  \n\nI remember that from our earlier conversation. 😊  \n\nIs there anything specific you'd like to talk about regarding your work as a Chief AI Engineer?  Perhaps a project you're particularly excited about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 99, 'total_tokens': 159, 'completion_time': 0.109090909, 'prompt_time': 0.002697227, 'queue_time': 0.249651663, 'total_time': 0.111788136}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1b69c4ec-7742-4afd-a08a-83f161731f49-0', usage_metadata={'input_tokens': 99, 'output_tokens': 60, 'total_tokens': 159})

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [5]:
config={"configurable":{"session_id":"chat1"}}

In [6]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Bedanta and I am a Chief AI Engineer")],
    config=config
)

In [7]:
response.content

"Hi Bedanta,\n\nIt's nice to meet you! That's a fascinating title. As a large language model, I'm always interested in learning more about how people are using AI in their work.\n\nWhat kind of projects are you working on as Chief AI Engineer?\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config
)
response

AIMessage(content="Bedanta.  \n\nI'm ready for a new topic if you are!  What else is on your mind? 😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 300, 'total_tokens': 330, 'completion_time': 0.054545455, 'prompt_time': 0.00618406, 'queue_time': 0.25620561000000003, 'total_time': 0.060729515}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b8b0e160-4084-42f2-9023-65b26944b04e-0', usage_metadata={'input_tokens': 300, 'output_tokens': 30, 'total_tokens': 330})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="message")
    ]
)
chain = prompt | model


In [30]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [31]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Bedanta and I am a Chief AI Engineer")],
    config=config
)

In [32]:
response=with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config
)
response.content

'Your name is Bedanta. You introduced yourself at the beginning of our conversation!  😊  \n\nHow can I help you further?\n'

In [34]:
#adding some complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [35]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [37]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Bedanta")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Bedanta! \n\nमुझे बहुत खुशी है आपसे मिलकर। मैं आपकी कैसे मदद कर सकता हूँ? \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [38]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

/Users/bedantaroy/Desktop/new_Langchain/newvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [39]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences.\n\nWhat's your favorite flavor? 😊🍦\n"

In [40]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2".  😊 \n\n\n\nLet me know if you have any other questions!\n'

In [41]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [42]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or personal information about you, so I don't know your name. 😊 \n\nWould you like to tell me your name?\n"